In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName("Aula Interativa 2 - ML") \
        .getOrCreate()

spark.version

23/01/31 02:28:41 WARN Utils: Your hostname, Deboras-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.68.104 instead (on interface en0)
23/01/31 02:28:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/31 02:28:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/31 02:28:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/01/31 02:28:41 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


'3.3.1'

In [2]:
titanic_df = spark.read.csv('/home/pcalais/XPE/cientista-dados/aula2/titanic.csv', header='True', inferSchema='True')

titanic_df.printSchema()

AnalysisException: Path does not exist: file:/home/pcalais/XPE/cientista-dados/aula2/titanic.csv

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['Age', 'Pclass', 'Fare'], outputCol='features')


In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

classifier = DecisionTreeClassifier(labelCol='Survived', featuresCol='features')

In [ ]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[assembler, classifier])

In [ ]:
train_data, test_data = titanic_df.randomSplit([0.7, 0.3])

In [ ]:
predictSurvivedModel = pipeline.fit(train_data)

In [ ]:
mean_age = titanic_df.agg({'Age': 'mean'}).collect()[0][0]
mean_age

In [ ]:
titanic_df = titanic_df.fillna(mean_age, subset=['Age'])

In [ ]:
train_data, test_data = titanic_df.randomSplit([0.7, 0.3])
predictSurvivedModel = pipeline.fit(train_data)

In [ ]:
predictions = predictSurvivedModel.transform(test_data)
predictions.select('passengerId', 'rawPrediction', 'prediction').show(5)



In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol='Survived', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)

accuracy

